In [68]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/cleaned_merged_seasons.csv')

/var/folders/c2/y0gmhjgn52s4fjvr1c_0n5_c0000gp/T/ipykernel_10706/147618075.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/cleaned_merged_seasons.csv')


In [113]:
data

,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,...,2.0,0.0,0,0,0,0,55,False,0,1
1,1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,...,1.0,0.0,1,0,0,0,60,True,0,1
2,2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,...,3.0,23.0,2,0,0,0,80,True,0,1
3,3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,...,1.0,0.0,0,0,0,0,50,False,0,1
4,4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,...,1.0,0.0,1,0,0,0,45,True,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98397,98397,2021-22,Wilfred Ndidi,MID,Leicester,0,0,0,0,0.0,...,4.0,0.0,0,-202,22,224,48,True,0,38
98398,98398,2021-22,Matt Ritchie,DEF,Newcastle,0,0,3,0,0.0,...,1.0,0.0,1,143,396,253,49,False,0,38
98399,98399,2021-22,Nathan Redmond,MID,Southampton,0,0,5,0,0.0,...,4.0,0.0,2,455,683,228,59,False,0,38
98400,98400,2021-22,Mathew Ryan,GK,Brighton,0,0,0,0,0.0,...,3.0,0.0,0,-2,0,2,45,True,0,38


In [30]:
data[data['name'] == 'Aaron Cresswell'].tail(20)

,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
84669,84669,2021-22,Aaron Cresswell,DEF,West Ham,0,0,0,0,0.0,...,2.0,0.0,0,-43368,1541,44909,53,True,0,19
85130,85130,2021-22,Aaron Cresswell,DEF,West Ham,0,0,0,0,0.0,...,1.0,0.0,0,-26734,479,27213,53,False,0,20
85715,85715,2021-22,Aaron Cresswell,DEF,West Ham,0,0,0,0,0.0,...,2.0,0.0,0,-42234,1424,43658,53,False,0,21
85716,85716,2021-22,Aaron Cresswell,DEF,West Ham,0,0,27,1,24.1,...,2.0,8.0,6,-42234,1424,43658,53,True,0,21
86458,86458,2021-22,Aaron Cresswell,DEF,West Ham,1,2,36,0,71.1,...,2.0,0.0,6,-15046,19151,34197,53,True,0,22
87344,87344,2021-22,Aaron Cresswell,DEF,West Ham,0,0,7,0,1.3,...,1.0,0.0,2,14953,23520,8567,53,False,0,23
87981,87981,2021-22,Aaron Cresswell,DEF,West Ham,0,2,30,1,25.8,...,1.0,2.0,8,42511,53033,10522,53,True,0,24
88679,88679,2021-22,Aaron Cresswell,DEF,West Ham,0,0,0,0,3.3,...,2.0,20.0,0,20152,33926,13774,53,False,1,25
89681,89681,2021-22,Aaron Cresswell,DEF,West Ham,1,3,27,0,16.8,...,1.0,18.0,8,-6429,14809,21238,53,True,0,26
90219,90219,2021-22,Aaron Cresswell,DEF,West Ham,0,1,27,1,15.6,...,1.0,3.0,7,54923,65859,10936,53,True,0,27


In [29]:
data.columns

Index(['Unnamed: 0', 'season_x', 'name', 'position', 'team_x', 'assists',
       'bonus', 'bps', 'clean_sheets', 'creativity', 'element', 'fixture',
       'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'opp_team_name',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98402 entries, 0 to 98401
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         98402 non-null  int64  
 1   season_x           98402 non-null  object 
 2   name               98402 non-null  object 
 3   position           98402 non-null  object 
 4   team_x             48930 non-null  object 
 5   assists            98402 non-null  int64  
 6   bonus              98402 non-null  int64  
 7   bps                98402 non-null  int64  
 8   clean_sheets       98402 non-null  int64  
 9   creativity         98402 non-null  float64
 10  element            98402 non-null  int64  
 11  fixture            98402 non-null  int64  
 12  goals_conceded     98402 non-null  int64  
 13  goals_scored       98402 non-null  int64  
 14  ict_index          98402 non-null  float64
 15  influence          98402 non-null  float64
 16  kickoff_time       984

In [71]:
kane = data[data['name'] == 'Harry Kane']
kane.groupby(by = 'season_x').agg({'bps': 'sum', 'creativity': 'sum', 'assists': 'sum', 
                                 'clean_sheets': 'sum', 'threat': 'sum', 'value': 'max', 
                                 'yellow_cards': 'sum', 'red_cards': 'sum', 'penalties_missed': 'sum', 
                                 'penalties_saved': 'sum', 'total_points': 'sum'})

,bps,creativity,assists,clean_sheets,threat,value,yellow_cards,red_cards,penalties_missed,penalties_saved,total_points
season_x,,,,,,,,,,,
2016-17,920,548.4,7,14,1530.0,119,3,0,1,0,224
2017-18,781,502.1,2,15,2355.0,131,5,0,1,0,217
2018-19,561,430.0,6,9,1359.0,126,5,0,0,0,160
2019-20,587,391.0,2,6,1071.0,111,4,0,0,0,158
2020-21,880,659.1,14,12,1585.0,119,1,0,0,0,242
2021-22,657,629.1,11,13,1609.0,126,5,0,0,0,185
